# Comparativa de modelos predictivos para identificación de sentimientos

El análisis de sentimientos es una técnica que permite detectar el sentimiento subyacente de un fragmento de texto. Los sentimientos se suelen clasificar como: neutros, positivos y negativos. Existen diferentes maneras de analizar el texto para poder identificar el proposito o sentimiento del texto.

## ¿Por qué es importante el análisis de texto?

El análisis de sentimientos es esencial para múltiples tareas empresariales y retos del nuevo siglo. Mediante éste, ha sido posible mejorar en gran medida la atención al cliente y la automatización de procesos donde el lenguaje humano es altamente utilizado. Estos son algunos de los sectores donde es más usado el análisis de sentimientos:

- Sector comercial.
- Sector bancario.
- Sector social.
- Sector logistico.
- Sector de transporte.

Las aplicaciones del análisis de sentimientos son amplias y poderosas. La capacidad de extraer conocimientos de los datos sociales es una práctica que está siendo adoptada ampliamente por organizaciones de todo el mundo. El lenguaje humano es complejo. Enseñar a una máquina a analizar los diversos matices gramaticales, variaciones culturales, jergas y errores ortográficos que se producen en las menciones online es un proceso difícil. Enseñar a una máquina a comprender cómo el contexto puede afectar el tono es aún más difícil.

# Configuración de ambiente

El conjunto de datos elegido para este proyecto esta conformado por un conjunto de tweets, el sentimiento que expresa y aquellas palabras que soportan el sentimiento.

Los tweets fueron obtenidos de la plataforma "Data for Everyone" de Figure Eight. El conjunto de datos se titula Análisis de sentimiento: tweets de emoción en texto con etiquetas de sentimiento existentes. El conjunto de datos se encuentra licenciado como creative commons 4.0, licencia internacional. 

## ¿Qué archivos conforman el conjunto de datos?

- **train.csv:** Conjunto de datos de entrenamiento
- **test.csv:** Conjunto de datos de prueba

## Columnas del conjunto de datos

- textID: Identificador único por tweet
- text: Texto del tweet
- sentiment: Sentimiento del tweet
- selected_text: [Solo entrenamiento] Texto que soporta el sentimiento

Cada fila contiene el texto de un tweet y una etiqueta de opinión. En el conjunto de entrenamiento, se le proporciona una palabra o frase extraída del tweet (selected_text) que encapsula el sentimiento proporcionado.



Para comenzar con el proceso de análisis de datos, primero importamos las librerías que necesitaremos en todo el proceso.

In [ ]:
!pip install --upgrade plotly==4.0.0

In [ ]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
from sklearn.preprocessing import LabelEncoder

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Activation,Flatten,Embedding,GlobalAveragePooling1D,Dropout,LSTM,Conv1D
from tensorflow.keras.models import Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from wordcloud import WordCloud, STOPWORDS
import tensorflow as tf
import missingno as msno
from collections import defaultdict
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json

%matplotlib inline

Cargamos los datos de entrada proporcionados por tweet-sentiment-extraction

In [ ]:
train = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv")
test = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/test.csv")
target = train['sentiment']

In [ ]:
train.head(10)

Con los datos cargados, procedemos a realizar una analisis inicial de los datos para tratar de identificar posibles problemas o dificultades.

In [ ]:
train.info()

Podemos identificar que existen algunos datos nulos, procedemos a eliminarlos

In [ ]:
train.dropna(inplace=True)

In [ ]:
train.info()

# Análisis exploratorio

Con los datos listos podemos proceder al analisis exploratorio de los datos

In [ ]:
train.describe()

In [ ]:
temp = train.groupby('sentiment').count()['text'].reset_index().sort_values(by='text',ascending=False)
temp

In [ ]:
fig = plt.figure(figsize=(16,6))
ax = fig.add_subplot(1,2,1)
sns.countplot(x='sentiment',data=train)
plt.title('Distribución de los sentimientos para los datos de entrenamiento')
plt.xlabel('Sentimientos')
plt.ylabel('Cantidad')

ax = fig.add_subplot(1,2,2)
sns.countplot(x='sentiment',data=test)
plt.title('Distribución de los sentimientos para los datos de prueba')
plt.xlabel('Sentimientos')
plt.ylabel('Cantidad')

In [ ]:
fig = go.Figure(go.Funnelarea(text =temp.sentiment, values = temp.text))
fig.show()

Procedemos a hallar el indice de jaccard entre el texto del tweet y el texto seleccionado. También obtenemos algunas variables relevantes como: número de palabras en el texto, número de palabras en el texto seleccionado y diferencia de palabras.

In [ ]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
temp = train.copy()

results_jaccard=[]
for ind,row in temp.iterrows():
    sentence1 = row.text
    sentence2 = row.selected_text

    jaccard_score = jaccard(sentence1,sentence2)
    results_jaccard.append(jaccard_score)

In [ ]:
temp['jaccard_score'] = results_jaccard

In [ ]:
temp.head()

In [ ]:
temp['count_st'] = train['selected_text'].apply(lambda x:len(str(x).split())) 
temp['count_t'] = train['text'].apply(lambda x:len(str(x).split())) 
temp['difference_in_words'] = temp['count_t'] - temp['count_st']

In [ ]:
temp.head()

Contamos la cantidad de palabras que existe en el texto completo y el texto seleccionado para tener una idea de la distribución que siguen los datos de entrada. Esta apróximación nos da una idea de posibles soluciones a plantear.

En los siguientes graficos nos podemos dar una idea del tamaño del texto seleccionado (en caracteres) según el sentimiento. 

**Anotaciones**

- La distribución para los tweets negativos y positivos es casi la misma.
- El texto seleccionado en los tweets neutrales es considerablemente más grande, que en los demás casos.

In [ ]:
rug_text_one = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
                'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
                'u', 'v', 'w', 'x', 'y', 'z']

rug_text_two = ['aa', 'bb', 'cc', 'dd', 'ee', 'ff', 'gg', 'hh', 'ii', 'jj',
                'kk', 'll', 'mm', 'nn', 'oo', 'pp', 'qq', 'rr', 'ss', 'tt',
                'uu', 'vv', 'ww', 'xx', 'yy', 'zz']
group_labels = ['Selected_Text', 'Text']
rug_text = [rug_text_one, rug_text_two] # for hover in rug plot
colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

# Create distplot with custom bin_size
fig = ff.create_distplot([temp['count_st'], temp['count_t']], group_labels,show_curve=False, rug_text=rug_text, colors=colors)
fig.update_layout(title_text='Distribución de número de palabras')
fig.update_layout(
    autosize=False,
    width=900,
    height=700,
    paper_bgcolor="LightSteelBlue",
)

fig.show()

In [ ]:
vals=[]
sent = ['positive', 'negative', 'neutral']
for i in range(0,3):
    x=temp[temp['sentiment']==sent[i]]['selected_text'].dropna().str.len()
    vals.append(x)

fig = ff.create_distplot(vals, sent)
fig.update_layout(title="Distribución del número de caracteres en la variable Selected text")
fig.show()

En los siguientes graficos podemos identificar que la distribución del número de palabras sigue un comportamiento similar a la distribución del número de caracteres.

El texto seleccionado en los tweets negativos y positivos presenta un claro sesgo hacia la izquierda, mientras para los tweets neutrales la distribución se mantiene muy similar al texto completo.

In [ ]:
sent=temp.sentiment.unique()
colors = ['k', 'b', 'r']
fig = plt.figure(figsize=(20,10))
for i in range(0,3):
    ax = fig.add_subplot(1,3,i+1)
    plt.hist(temp[temp['sentiment']==sent[i]]['count_t'], color=colors[i])
    plt.title(sent[i])
fig.suptitle("Ditribución de número de palabras por tweets", fontsize=14)

In [ ]:
sent=temp.sentiment.unique()
colors = ['k', 'b', 'r']
fig = plt.figure(figsize=(20,10))
for i in range(0,3):
    ax = fig.add_subplot(1,3,i+1)
    plt.hist(temp[temp['sentiment']==sent[i]]['count_st'], color=colors[i])
    plt.title(sent[i])
fig.suptitle("Ditribución de número de palabras por tweets", fontsize=14)

**Las siguientes graficas nos permiten visualizar que el indice de Jaccard para los tweets neutrales en su mayoría es 1, es decir, la cantidad de palabras del texto seleccionado es la misma cantidad de palabras en el texto completo del tweet.**

En los sentimientos positivos y negativos, la distribución kernel del indice de Jaccard muestra dos nucleos principales de concentración de la palabras. El nucleo cercano a 1, nos da indicios que los tweets con muy pocas palabras siempre tienen el mismo número de palabras seleccionadas. Lo cual resulta bastante útil para la clasificación.

Es interesante análizar la aproximación kernel del indice de Jaccard y de la diferencia de palabras. Pues, junto con los datos obtenidos anteriormente de la distribución de palabras seleccionadas y la distribución de palabras totales, nos pueden dar una idea del comportamiento del indice de Jaccard esperado (metrica de validación) según el sentimiento.


In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(temp[temp['sentiment']=='positive']['difference_in_words'], shade=True, color="b").set_title('Distribución Kernel de diferencia de palabras')
p2=sns.kdeplot(temp[temp['sentiment']=='negative']['difference_in_words'], shade=True, color="r")

In [ ]:
plt.figure(figsize=(12,6))
p1=sns.kdeplot(temp[temp['sentiment']=='positive']['jaccard_score'], shade=True, color="b").set_title('Distribución Kernel de diferencia de palabras')
p2=sns.kdeplot(temp[temp['sentiment']=='negative']['jaccard_score'], shade=True, color="r")

# Análisis de palabras

Para continuar con el análisis de palabras es necesario limpiar los tweets de:

- Signos de puntuación.
- Mayúsculas (Todo el texto se procesará en minúscula).
- Información entre parentesis y brakets.
- Números.
- Palabras vácias

In [ ]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
train_clean = train.copy()

train_clean['text'] = train['text'].apply(lambda x:clean_text(x))
train_clean['selected_text'] = train['selected_text'].apply(lambda x:clean_text(x))

In [ ]:
train_clean.head()

In [ ]:
train_clean['temp_list'] = train_clean['selected_text'].apply(lambda x:str(x).split())
top = Counter([item for sublist in train_clean['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['palabra_comun','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp, x="count", y="palabra_comun", title='Las palabras más comunes en el texto seleccionado', orientation='h', 
             width=700, height=700,color='palabra_comun')
fig.show()

Dado que hay muchas palabaras vácias (Palabras sin significado de manera individual). Se debe proceder a limpiar el texto seleccionado para identificar las palabaras que mas se repiten.

In [ ]:
def remove_stopword(x):
    return [y for y in x if y not in stopwords.words('english')]
train_clean['temp_list'] = train_clean['temp_list'].apply(lambda x:remove_stopword(x))

In [ ]:
top = Counter([item for sublist in train_clean['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Purples')

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Las palabras más comunes en el texto seleccionado SIN stopwords', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
train_clean['temp_list1'] = train_clean['text'].apply(lambda x:str(x).split())
train_clean['temp_list1'] = train_clean['temp_list1'].apply(lambda x:remove_stopword(x)) 

In [ ]:
top = Counter([item for sublist in train_clean['temp_list1'] for item in sublist])
temp = pd.DataFrame(top.most_common(25))
temp = temp.iloc[1:,:]
temp.columns = ['palabra_comun','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp, x="count", y="palabra_comun", title='Palabras cómunes en los tweets', orientation='h', 
             width=700, height=700,color='palabra_comun')
fig.show()

## División por sentimientos.

In [ ]:
Positive_sent = train_clean[train_clean['sentiment']=='positive']
Negative_sent = train_clean[train_clean['sentiment']=='negative']
Neutral_sent = train_clean[train_clean['sentiment']=='neutral']

In [ ]:
top = Counter([item for sublist in Positive_sent['temp_list'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['palabra_comun','count']
temp_positive.style.background_gradient(cmap='Greens')

In [ ]:
top = Counter([item for sublist in Negative_sent['temp_list'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words','count']
temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
top = Counter([item for sublist in Neutral_sent['temp_list'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Greys')

Es interesante resaltar que algunas palabras se repiten en las tres divisiones (get,go,dont,got,u,cant,lol,like). Esto a simple vista puede parecer un error de clasificación, sin embargo, solo es una muestra de lo nutrido que se encuentra nuestro conjunto de datos.

# Conclusiones EDA y Análisis de palabras

- Es posible visualizar que las palabras seleccionadas para sustentar el sentimiento, siempre se encuentran dentro del texto principal.

- Las palabras seleccionadas siempre son menores o iguales en cantidad a las palabras que contiene el texto principal del tweet.

- En las palabras seleccionadas con sentimiento neutro existe una tendencia clara a que la cantidad de palabras seleccionadas tengan un tamaño igual al tamaño del texto principal.

- La distribución de la cantidad de palabras de los tweet con sentimiento positivo y negativo son más complicadas de identificar y están distribuidas en dos secciones principalmente. Aquellas que contiene todas las palabras y aquellas que contiene aproximadamente una cuarta parte de las palabras del texto principal.

- Las aproximaciones más comunes para este tipo de problemas son las Question And Answers (QA).

# Análisis de sentimientos

El análisis de sentimientos es el proceso que permite comprender una opinión sobre un tema escrito o hablado, en un idioma en particular. En el enfoque computacional, el análisis de sentimientos usan herramientas de Natural Language Processing (NLP), análisis de textos y computación linguistica para enfrentar problemas que requieran entender los sentimientos.

## ¿Cómo se debería enfrentar un problema de este estilo?

1. Recopilar el texto que se desea analizar.
2. Dividir el texto en palabras.
3. Tokenizar las palabras.
4. Transformar en vectores las palabra

Para cualquier algoritmo de aprendizaje automático o aprendizaje profundo, se necesita que las entradas sean valores escalares o matrices de valores escalares. Normalmente se toman las palabras y se convierten en vectores. A cada palabra se le asigna un valor único de modo que cada vector de la palabra representa su contexto, significado y semántica.


De esta manera, las palabras con significado y contexto similares están en el mismo espacio vectorial. Una palabra que no tiene similitud se expresa en un ángulo de 90 grados. Las palabras con total similitud se colocan cerca del ángulo de 0 grados. En conclusión los vectores de las palabras son importantes porque nos permiten identificar la cercanía que pueden tener las palabras según su naturaleza.

## Selección de herramientas para clasificación

A partir del EDA podemos identificar que el problema que encontramos en nuestro proyecto se puede afrontar mediante un análisis de sentimientos basado en Question And Answer (QA), el cual, consiste en diseñar modelos que permitan encontrar una respuesta dentro un contexto definido, es decir, dentro de un texto determinado, encontrar una respuesta basado en una pregunta.

En nuestro caso, el contexto hace referencia al Tweet, la respuesta sería el texto seleccionado y la pregunta el sentimiento asociado.

Normalmente para este tipo de problemas se útilizan soluciones de embeddings con contexto como los propuestos por BERT o ELMO, combinados con algoritmos de machine learning. Sin embargo, para esta monografia también se quiere probar un enfoque más clásico combinando embedding sin contexto como Glove y algoritmos de machine learning. 

Antes de comenzar a plantear la solución, se realizará como prueba inicial un análisis de sentimientos basado en Glove para identificar si vale la pena plantear este tipo de solución o proponemos modelos tipo BERT unicamente.

### Aproximación - identificación de sentimientos mediante GloVe

In [ ]:
NB_START_EPOCHS = 10  # Número de iteraciones de entrenamiento
BATCH_SIZE = 512  # Tamaño de grupos usados en entrenamiento
GLOVE_DIM = 100  # Número de dimensiones los embeddings GloVe

El siguiente proceso de limpieza es menos agresivo que el realizado para nuestros datos iniciales. Este proceso mantiene más información y le permite a algoritmos como BERT mantener información que podrías ser relevante.

In [ ]:
#Reference https://www.kaggle.com/parulpandey/eda-and-preprocessing-for-bert

def clean(tweet):
  #Covert to lowercase
  tweet=tweet.lower()

  #Remove html tags
  tweet=re.sub('<.*?>','',tweet)

  #Remove text in square brackets
  tweet=re.sub('\[.*?\]','',tweet)

  #Remove hyperlinks
  tweet=re.sub('https?://\S+|www\.\S+','',tweet)

  
  return tweet

Hallamos la cantidad de palabras que cubran la mayor parte de los datos, para esto contamos las palabras e identificamos el cuantil de apróximadamente 99.9999, tratando de apróximarlo a 100.

In [ ]:
train_temp = train.copy()
train_temp['count'] = train_temp['selected_text'].apply(lambda x:len(str(x).split()))

In [ ]:
print(train_temp['count'].quantile(0.9999))
print(train_temp['count'].quantile(0.99999))
print(train_temp['count'].quantile(0.999999))
print(train_temp['count'].quantile(0.9999999))

In [ ]:
seq_length = 33

In [ ]:
#Cleaning the text attribute
train_sent = train.copy()
train_sent['clean_text']=train_sent['text'].apply(clean)
train_sent['clean_selected_text']=train_sent['selected_text'].apply(str.lower)
train_sent = train_sent.reset_index()
train_sent.head()

Dividimos los datos de entreamiento en dos grupos, los datos que nos van a permitir entrenar el modelo y los datos que nos van a permitir probar los resultados.

In [ ]:
from sklearn.model_selection import train_test_split


X_train,X_test,Y_train,Y_test=train_test_split(train_sent['clean_text'],train_sent['sentiment'],
                                               test_size=0.2,random_state=17, stratify=train_sent['sentiment'])


print('X_train shape',X_train.shape,' Y_train shape ',Y_train.shape)
print('X_test shape',X_test.shape,' Y_test shape ',Y_test.shape)

Tokenizamos las frases para que sean representados por enteros y puedan ser procesadas por los algoritmos de machine learning

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='UNK',filters='')
tokenizer.fit_on_texts(X_train)

X_train_clean_text = tokenizer.texts_to_sequences(X_train)
X_train_tokens = tf.keras.preprocessing.sequence.pad_sequences(X_train_clean_text,maxlen=seq_length,padding='post')

X_test_clean_text = tokenizer.texts_to_sequences(X_test)
X_test_tokens = tf.keras.preprocessing.sequence.pad_sequences(X_test_clean_text,maxlen=seq_length,padding='post')

Codificamos los sentimientos para que sean representados por vectores

In [ ]:
from keras.utils.np_utils import to_categorical

le = LabelEncoder()
y_train_le = le.fit_transform(Y_train)
y_test_le = le.transform(Y_test)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)

display(y_train_oh)
display(y_test_oh)

Generamos un tercer grupo de datos para ayudar al algoritmo de la red neural de keras a validar los datos. Estos datos son tomados de los datos de entrenamiento.

In [ ]:
X_train_emb, X_valid_emb, y_train_emb, y_valid_emb = train_test_split(X_train_tokens, y_train_oh, test_size=0.2, random_state=37)

In [ ]:
embeddings_index = {}
f = open('/kaggle/input/glove-global-vectors-for-word-representation/glove.twitter.27B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('%s Vectores de palabras encontrados.' % len(embeddings_index))

In [ ]:
vocab_size=len(tokenizer.word_index)+1
print(vocab_size)

In [ ]:
embedding_matrix = np.zeros((vocab_size, GLOVE_DIM ))
for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:    
    embedding_matrix[i] = embedding_vector

Se toma la desición de usar la siguiente arquitectura para generar una red sencilla que permita verificar los resultados generales que se puede tener de una apróximación de embedding tipo Glove (sin contexto) para identificar los sentimientos de unos tweets

**Anotaciones**

- Se usará el algoritmo de optimización Adam, pues permite mejorar los tiempos de procesamiento, manteniendo resultados aceptables. Algunas de sus caracteristicas son: Fácil de implementar, computacionalmente eficiente, requisitos de memoria pequeños, muy adecuado para problemas con grandes cantidades de datos y parámetros, apropiado para problemas con gradientes muy ruidosos o dispersos.
- La metrica seleccionada para análizar la viabilidad del modelo fue la presición.
- Es posible mejorar los resultados a variar el tamaño del BATCH de entrenamiento.
- La función de perdida elegida fue la entropia cruzada pues es bastante recomendada para problemas donde existen dos o más etiquetas. Es decir, problemas de clasificación multiple.

In [ ]:
input_layer=Input((seq_length,),name='input')
embedding_layer=Embedding(vocab_size, GLOVE_DIM ,weights=[embedding_matrix],input_length=seq_length,trainable=False)(input_layer)

conv1d=Conv1D(6,2,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=20),name='conv1d')(embedding_layer)
dropout=Dropout(0.2,name="drop_out")(conv1d)
flatten=Flatten(name='flatten')(dropout)

output=Dense(3,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_uniform(seed=45),name='output')(flatten)

simpleNeural=Model(inputs=[input_layer],outputs=[output])

simpleNeural.summary()

In [ ]:
simpleNeural.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
simpleNeural.fit(X_train_emb, y_train_emb,batch_size=BATCH_SIZE ,epochs=NB_START_EPOCHS ,
           validation_data=(X_valid_emb, y_valid_emb))

In [ ]:
results = simpleNeural.evaluate(X_test_tokens, y_test_oh)
print(results)

In [ ]:
import numpy

Y_test_predict = simpleNeural.predict(X_test_tokens)
predicted_classes = numpy.argmax(Y_test_predict, axis=1)
predicted_classes

In [ ]:
from sklearn.metrics import confusion_matrix

Y_test_predict = simpleNeural.predict(X_test_tokens)
predicted_classes = numpy.argmax(Y_test_predict, axis=1)
confusion_matrix(y_true=y_test_le, y_pred=predicted_classes, normalize='true')


Los mejores resultados se encuentran en la clasificación de los tweets con sentimientos neutrales. Lo cual es posible visualizarse en la matriz de confusión presentada anteriormente.

#### Arquitectura de red neuronal con capa densa.

In [ ]:
input_layer=Input((seq_length,),name='input')
embedding_layer=Embedding(vocab_size,100,weights=[embedding_matrix],input_length=seq_length,trainable=False)(input_layer)
flatten=Flatten(name='flatten')(embedding_layer)
output=Dense(3,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_uniform(seed=45),name='output')(flatten)

simpleNeuralDen=Model(inputs=[input_layer],outputs=[output])

simpleNeuralDen.summary()

In [ ]:
simpleNeuralDen.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
simpleNeuralDen.fit(X_train_emb, y_train_emb,batch_size=BATCH_SIZE,epochs=NB_START_EPOCHS,
           validation_data=(X_valid_emb, y_valid_emb))

In [ ]:
results = simpleNeuralDen.evaluate(X_test_tokens, y_test_oh)
print(results)

In [ ]:
import numpy

Y_test_predict = simpleNeuralDen.predict(X_test_tokens)
predicted_classes = numpy.argmax(Y_test_predict, axis=1)
predicted_classes

In [ ]:
from sklearn.metrics import confusion_matrix

Y_test_predict = simpleNeuralDen.predict(X_test_tokens)
predicted_classes = numpy.argmax(Y_test_predict, axis=1)
confusion_matrix(y_true=y_test_le, y_pred=predicted_classes, normalize='true')

## Conclusiones aproximaciones

- Mediante metodologías de embbedings sin contexto como Glove es posible obtener resultados interesantes como punto de pantida para una análisis más extenso.
- Existe un error considerable en los tweets neutrales que puede eliminarse al procesarlos bajo la luz de los resultados obtenidos en el EDA, es decir, para los tweets con sentimiento neutral se puede retornar inmediatamente el mismo valor de entrada.
- El resultado con una capa convolucional para el caso de un modelo con redes neuronales brinda mejores resultados.